In [6]:
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

# Assuming your GeoDataFrame is named 'gdf' and already in long format
# with columns: GEOID, acs_year, indexed_vulnerability, geometry, etc.
eaz = gpd.read_file('atx_eaz_multi_year.geojson')

# Convert to GeoJSON for Plotly
geojson = eaz.__geo_interface__

In [ ]:
# Create Plotly figure
fig = go.Figure()

# Add chloropleth traces for each year
for year in eaz['acs_year'].unique():
    year_data = eaz[eaz['acs_year'] == year]

    if year == 2019:    # Starting year
        hovertemplate = (
            "<b>%{customdata[0]}</b><br>" +
            "<b>Year:</b> %{customdata[1]}<br>" +
            "<b>Vulnerability Index:</b> %{z:.2f}<br>" +
            "<extra></extra>"
        )
    else:
        hovertemplate = (
            "<b>%{customdata[0]}</b><br>" +
            "<b>Year:</b> %{customdata[1]}<br>" +
            "<b>Vulnerability Index:</b> %{z:.2f}<br>" +
            "<b>Difference from 2019:</b> %{customdata[2]:+.2f}<br>" +
            "<extra></extra>"
        )

    # Preparing customdata
    customdata = year_data[['NAME', 'acs_year', 'difference_since_2019']].copy()
    customdata['difference_since_2019'] = customdata['difference_since_2019'].apply(
        lambda x: f"{x:+.2f}" if pd.notnull(x) else "N/A"
    )
    
    fig.add_trace(go.Choroplethmapbox(
        geojson=geojson,
        locations=year_data.index,
        z=year_data['indexed_vulnerability'],
        colorscale="RdYlGn_r",
        zmin=eaz['indexed_vulnerability'].min(),
        zmax=eaz['indexed_vulnerability'].max(),
        marker_opacity=0.8,
        marker_line_width=0.25,
        visible=(year == eaz['acs_year'].min()),  # Only the first year visible initially
        name=str(year),  # Add name for the slider
        hovertemplate=hovertemplate,
        customdata=customdata[['NAME', 'acs_year', 'difference_since_2019']],
        colorbar=dict(
            thickness=15,
            outlinecolor='rgba(0,0,0,0)',  # Transparent outline
            bgcolor='rgba(255,255,255,0.5)'  # Semi-transparent background
        )
    ))

# Update layout
fig.update_layout(
    title_text="Indexed Vulnerability - 2019",  # Initialize title here
    mapbox_style="carto-positron",
    width=1200,  # Set the desired width
    height=800,  # Set the desired height
    mapbox_zoom=8,
    mapbox_center={"lat": 30.2672, "lon": -97.7431},
    sliders=[dict(
        active=0,
        steps=[dict(
            label=str(year),
            method="update",
            args=[{"visible": [y == year for y in eaz['acs_year'].unique()]},
                  {"title": f"Indexed Vulnerability - {year}"}],    # Updates the title
        ) for year in eaz['acs_year'].unique()],
        currentvalue=dict(
            prefix="Selected year: ",
            font=dict(size=10, color="black"),
            visible=True,
            xanchor='left'
        )
    )]
)

# Show the figure
fig.show()

In [40]:
# Exporting to HTML
fig.write_html("eaz_map.html")